This notebook was used to create and debug functionality for Thellier GUI in 3.0.  

## Getting started

In [1]:
import pmagpy.new_builder as nb
from pmagpy import ipmag
import os
import json
import numpy as np
import sys

import pandas as pd
from pandas import DataFrame
from pmagpy import validate_upload3 as validate_upload
from pmagpy import pmag

reload(nb)

WD = os.path.join("..", "3_0", "Megiddo")


## Working with anisotropy data for Thellier GUI

In [2]:
# get specimen level data from specimen table
#contribution = nb.Contribution(WD, read_tables=['specimens', 'samples','sites'])
contribution = nb.Contribution(WD, custom_filenames={'samples': 'new_samples.txt','sites': 'new_sites.txt',
                                                     'specimens': 'new_specimens.txt'})
if 'specimens' in contribution.tables:
    spec_container = contribution.tables['specimens']
    spec_data = spec_container.df
    
print spec_data.columns
spec_container.write_magic_file(custom_name='test_spec.txt',dir_path=WD)

-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org
-I- Importing suggested vocabularies from https://earthref.org
-W- No such file: /Users/nebula/Python/PmagPy/data_files/3_0/Megiddo/images.txt
Index([u'aniso_alt', u'aniso_ftest', u'aniso_ftest12', u'aniso_ftest23',
       u'aniso_s', u'aniso_s_n_measurements', u'aniso_s_sigma', u'aniso_type',
       u'aniso_v1', u'aniso_v2', u'aniso_v3', u'citations', u'description',
       u'dir_comp', u'dir_dang', u'dir_dec', u'dir_inc', u'dir_mad_free',
       u'dir_n_comps', u'dir_n_measurements', u'dir_tilt_correction',
       u'experiments', u'geologic_classes', u'geologic_types', u'int_abs',
       u'int_b_beta', u'int_corr', u'int_corr_anisotropy',
       u'int_corr_cooling_rate', u'int_dang', u'int_drats', u'int_f',
       u'int_fvds', u'int_g', u'int_mad_free', u'int_n_measurements',
       u'int_n_ptrm', u'int_q', u'int_treat_dc_field', u'lithologies',
       u'meas_step_max', u'meas_s

In [3]:
# find the record of a particular specimen.  if it already has aniso data, then modify in place
# otherwise add a new record and modify that....    
specimen = 'hz05a1' # a specimen with aniso data
new_s = '.3:.3:.3:0:0:0'

# add numeric index column temporarily
spec_data['num'] = range(len(spec_data))




In [4]:

# iterate through specimen names and update or add anisotropy data
for spec in [specimen, "mc01a"]:
    
    # match specimen name
    cond1 = spec_data['specimen'].str.contains(spec) == True
    # anisotropy data is present
    cond2 = spec_data['aniso_type'].str.contains('ATRM')==True
    #cond3 = spec_data['aniso_s'].notnull() == True
    condition = (cond1 & cond2)
    if len(spec_data[condition]) > 0:
        print 'update existing record', spec
        # get list of all rows for which both conditions are true
        #inds = spec_data[condition]['num'].values.tolist()
        inds = spec_data[condition]['num']
        print inds
        # in case there are multiple anisotropy rows for a specimen, update each row
            # get pre-existing data from that row
        existing_data = dict(spec_data.iloc[inds[0]])
        print existing_data['aniso_s']
            # add to existing_data with values from new_data
        new_data = {'aniso_s': new_s, 'specimen': spec}
        existing_data.update(new_data)
            # update row
        spec_container.update_row(inds[0], existing_data)
        if len(inds)>1: # more than one meeting condition
            for ind in inds[1:]: 
                #ind=ind.values.tolist()
                spec_container.delete_row(ind)

                  
    else:
        print 'no record found - creating new one', spec
        # add new row
        spec_container.add_row(spec, {'aniso_s': new_s, 'specimen': spec})


# sort so that all rows for a specimen are together
spec_data.sort_index(inplace=True)
# redo temporary index
spec_data['num'] = range(len(spec_data))
# see all records for mc01a & mc121d1 
# now includes 1 new anisotropy record for mc01a, and 1 updated record for mc121d1
spec_data.loc[['mc01a', specimen]][['specimen', 'aniso_s', 'num']]

update existing record hz05a1
specimen
hz05a1    1
Name: num, dtype: int64
0.344050 : 0.328649 : 0.327301 : -0.009684 : -0.000697 : -0.000627
no record found - creating new one mc01a


,specimen,aniso_s,num
specimen,,,
mc01a,mc01a,.3:.3:.3:0:0:0,373
hz05a1,hz05a1,None,0
hz05a1,hz05a1,.3:.3:.3:0:0:0,1


In [5]:
def flip_dict(dict):
    newdict={}
    for key in dict:
        val=dict[key]
        newdict[val]=key
    return newdict

In [6]:
spec3_spec2={'int_drats': 'specimen_drats', 'site': 'er_site_name', 'int_mad': 'specimen_int_mad', 'sample': 'er_sample_name', 'measurement_step_max': 'meas_step_max', 'specimen_n': 'dir_n_measurements', 'int_n_measurements': 'specimen_int_n', 'int_corr': 'specimen_correction', 'int_rsc': 'specimen_rsc', 'analyst_names': 'er_analyst_mail_names', 'int_scat': 'specimen_scat', 'int_ptrm_n': 'specimen_int_ptrm_n', 'citations': 'er_citation_names', 'int_gmax': 'specimen_gmax', 'int_dang': 'specimen_int_dang', 'dir_tilt_correction': 'specimen_tilt_correction', 'location': 'er_location_name', 'dir_comp': 'specimen_comp_name', 'specimen_magn_moment': 'magn_moment', 'int_w': 'specimen_w', 'specimen': 'er_specimen_name', 'int_q': 'specimen_q', 'int_fvds': 'specimen_fvds', 'specimen_mad': 'dir_mad_free', 'int_frac': 'specimen_frac', 'meas_step_min': 'measurement_step_min', 'int_f': 'specimen_f', 'software_packages': 'magic_software_packages', 'dir_mad_free': 'specimen_mad', 'magn_moment': 'specimen_magn_moment', 'instrument_codes': 'magic_instrument_codes', 'int_b_beta': 'specimen_b_beta', 'dir_n_comps': 'specimen_comp_n', 'int_md': 'specimen_md', 'dir_n_measurements': 'specimen_n', 'dir_inc': 'specimen_inc', 'specimen_magn_volumn': 'magn_volumn', 'meas_step_max': 'measurement_step_max', 'dir_alpha95': 'specimen_alpha95', 'magn_volumne': 'specimen_magn_volumn', 'measurement_step_min': 'meas_step_min', 'meas_step_unit': 'measurement_step_unit', 'dir_dec': 'specimen_dec', 'method_codes': 'magic_method_codes', 'result_quality': 'specimen_flag', 'dir_dang': 'specimen_dang'} 
site3_site2={'int_abs_sigma' : 'site_int_sigma', 'int_abs_sigma_perc' : 'site_int_sigma_perc', 'int_n_samples' : 'site_int_n', 'dir_alpha95' : 'site_alpha95', 'dir_k' : 'site_k', 'dir_n_samples' : 'site_n', 'dir_n_specimens_lines' : 'site_n_lines', 'dir_n_specimens_planes' : 'site_n_planes', 'dir_r' : 'site_r'}
aniso3_aniso2={'specimen':'er_specimen_name', 'aniso_type':'anisotropy_type', 'description':'result_description', 'aniso_ftest':'anisotropy_ftest', 'aniso_ftest12':'anisotropy_ftest12', 'aniso_ftest23':'anisotropy_ftest23', 'aniso_s_mean':'anisotropy_mean', 'aniso_s_n_measurements':'anisotropy_n', 'aniso_s_sigma':'anisotropy_sigma', 'aniso_s_unit':'anisotropy_unit', 'aniso_tilt_correction':'anisotropy_tilt_correction'}

samp3_samp2 = {'int_n_specimens' : 'sample_int_n', 'int_abs_sigma' : 'sample_int_sigma', 'int_abs_sigma_perc' : 'sample_int_sigma_perc', 'dir_alpha95' : 'sample_alpha95', 'dir_n_specimens' : 'sample_n', 'dir_n_specimens_lines' : 'sample_n_lines', 'dir_n_specimens_planes' : 'sample_n_planes', 'dir_k' : 'sample_k', 'dir_r' : 'sample_r'}



print flip_dict(site3_site2)


{'site_int_sigma': 'int_abs_sigma', 'site_alpha95': 'dir_alpha95', 'site_n': 'dir_n_samples', 'site_k': 'dir_k', 'site_int_n': 'int_n_samples', 'site_int_sigma_perc': 'int_abs_sigma_perc', 'site_r': 'dir_r', 'site_n_planes': 'dir_n_specimens_planes', 'site_n_lines': 'dir_n_specimens_lines'}
